# ENGRI 1120: Parallel Cell-free Production and Recovery of the mRNA BNT-162b2 Vaccine

Team #7, Vivian Liu (vl274), Megan Leverock (mrl257), William Manno (wjm252), Francis Pham (fdp25)

<img src="figs/CHEMEFINAL.png" style="width:100%">

## Introduction

The creation of the Covid-19 vaccine was no simple feat especially with the time frame it was developed in. This vaccine is unlike traditional vaccines - while traditional vaccines inject a weakened or dead virus into our bodies, the messenger RNA vaccine injects mRNA into our body and then teaches cells to make a protein that triggers an immune response once someone is infected with COVID-19. Now, Olin Engineering has been contracted to design a system that mass produces a variant of the Covid-19 G-BNT-162b2 mRNA vaccine. Our apparatus uses PURExpress in-vitro expression. PURExpress is a reconstructed protein synthesis system where all necessary components needed for in-vitro are derived from purified E-Coli. The PURExpress will express mRNA molecules needed for the creation of the vaccine. Once synthesized and injected into an individual, the vaccine will stimulate the body’s immune response against the Covid variant. This vaccine will help your body fight off the Covid variant faster, more effectively, and therefore will slow the spread of the virus.

Our engineering team decided to arrange reactor chips in a parallel configuration. Although the series configuration requires less chips and works more efficiently, the bounds and the conditions are chip dependent. The parallel configuration is much simpler since the chips are uniform and the environment within each chip is the same. Our system contains two syringe pumps. The first pump that is pumping in stream 1 has a flow rate of 0.0015 L/min and contains G-BNT-162b2 set at a concentration of 2000 μmol/L. The second pump is pumping in stream 2 at a flow rate of  0.045 L/min and contains PURExpress. The PURExpress contained T7 RNA polymerase at a concentration of 100.0 μmol/L and M_atp_c, M_utp_c, M_ctp_c, M_gtp_c all at the same concentration of 100000 μmol/L. These two streams are heated to 37°C through heat exchangers and are then fed into two splitters. The splitters then split the streams into 15 chips where the reaction takes place. Then, from these 15 chips, 15 streams enter into the mixer. After the mixer, the stream enters through another heat exchanger to cool the solution below -25° C and then goes to 7 levels of MSUs working at 95% efficiency producing a 99.76% pure mRNA. 

The syringe pumps we are using to pump in PURExpress and G-BNT-162b2 cost \\$1000 each so we are spending \\$2000 in total for our pumps. The cost of PURExpress wass \\$0.0095 per μmol. With a volumetric flow rate of 45 mL/min operating for 525,600 minutes a year, the cost of PURExpress came out to be \\$224, 694, 000. Our apparatus has three heat exchangers in use, each costing \\$75, with a total cost of \\$225. The stream splitters, as well as the mixing unit, are free of cost and therefore have no impact on total cost. Our system has a total of 15 chips each costing \\$100, producing a total cost of \\$1500. Then moving onto our separation unit, we use a linear separation with 7 levels of separation and each MSU costs \\$100 giving a total cost of \\$700. The total cost of our system for the year is \\$37423.54.

## Materials and Methods and Assumptions 
  

Well mixed

Steady state

Open system

Ideal atmospheric pressure

All reaction cells are isothermal

Reaction cells are operating at optimum temperature 37° C


### Project Setup

In [372]:
import Pkg; Pkg.activate("."); Pkg.resolve(); Pkg.instantiate();

  Activating project at `c:\Users\trime\Cornell Semester 1\ENGRI 1120\ENGRI-Final\project`
  No Changes to `C:\Users\trime\Cornell Semester 1\ENGRI 1120\ENGRI-Final\project\Project.toml`
  No Changes to `C:\Users\trime\Cornell Semester 1\ENGRI 1120\ENGRI-Final\project\Manifest.toml`


In [373]:
# load reqd packages and set paths -
using JLD2
using FileIO
using PrettyTables
using DataFrames
using GLPK

# setup paths -
const _ROOT = pwd();
const _PATH_TO_DATA = joinpath(_ROOT, "data");

#### Load the project code library
The call to the `include` function loads the `ENGRI-1120-Project-CodeLib.jl` library into the notebook; the library contains functions we can use during the project. In particular, it includes the function:

* The `compute_optimal_extent(stoichiometric_matrix::Array{Float64,2}, default_bounds_array::Array{Float64,2},
    species_bounds_array::Array{Float64,2}, objective_coefficient_array::Array{Float64,1}; min_flag::Bool = true) -> Tuple` function calls the [GLPK](https://www.gnu.org/software/glpk/) linear program solver. The `results` tuple contains several things, but the important ones are `calculated_flux_array`, `objective_value`, and the status/exit flags `status_flag` and `exit_flag` (which let us know if the solver successfully found a solution).
* The `build(model::Type{MSULatticeModel}; ṅₒ::Float64, L::Int64, u::Float64, d::Float64) -> MSULatticeModel` function builds a [Binary tree](https://en.wikipedia.org/wiki/Binary_tree) of Magical Separation Units (MSUs). Arguments: $ṅₒ$ denotes the species mole flow rate into the separation system, $L$ denotes the number of layers of the tree, $u$ denotes the `up` factor (split for the `up` path), and $d$ denotes the `down` factor (the split for the `down` path). This function returns the `MSULatticeModel` model, which contains the column array `data` holding the species mole flow rate for each node in the tree. 
* The `build_nodes_dictionary(levels::Int64) -> Dict{Int64,Array{Int64,1}}` function constructs a dictionary of node indexes for each level of the tree; keys are the tree levels.

In [374]:
include("ENGRI-1120-Project-CodeLib.jl");

In [375]:
# load the model file -
model = load(joinpath(_PATH_TO_DATA, "ENGRI-1120-BNT162b2-Model.jld2"))["model"]

┌ Warning: the following fields are present in type DataFrame saved in the file but not present in the type the workspace:
│ 
│ metadata
│ colmetadata
│ allnotemetadata,
│ 
│ Data in these fields will not be accessible
└ @ JLD2 C:\Users\trime\.julia\packages\JLD2\k9Gt0\src\data\reconstructing_datatypes.jl:196


Dict{String, Any} with 7 entries:
  "stochiometric_matrix" => [-1.0 0.0 … 0.0 -1.0; -1.0 0.0 … -1.0 0.0; … ; 1.0 …
  "list_of_reactions"    => ["TX_BNT_162b2_binding", "TX_BNT_162b2_open", "BNT_…
  "reaction_table"       => 6×7 DataFrame…
  "flux_bounds_array"    => [-1000.0 1000.0; 0.0 1000.0; … ; 0.0 1000.0; 0.0 10…
  "mRNA_sequence"        => ['C', 'U', 'C', 'U', 'U', 'A', 'U', 'U', 'U', 'G'  …
  "list_of_species"      => ["G_BNT_162b2", "T7RNAP", "M_atp_c", "M_utp_c", "M_…
  "gene_sequence"        => ['G', 'A', 'G', 'A', 'A', 'T', 'A', 'A', 'A', 'C'  …

In [376]:
# get stuff from the model data structure -
S = model["stochiometric_matrix"]; # fix the spelling in the model file
flux_bounds_array = model["flux_bounds_array"];
list_of_species = model["list_of_species"];
list_of_reactions = model["list_of_reactions"];
reaction_table = model["reaction_table"];
gene_sequence = model["gene_sequence"];

In [377]:
reaction_table

,id,forward
,String,String
1,TX_BNT_162b2_binding,G_BNT_162b2+T7RNAP
2,TX_BNT_162b2_open,G_BNT_162b2_T7RNAP_closed
3,BNT_162b2_transcription,G_BNT_162b2_T7RNAP_open+798*M_atp_c+1004*M_utp_c+1060*M_ctp_c+1312*M_gtp_c
4,mRNA_BNT_162b2_degradation,mRNA_BNT_162b2
5,RNAP_deactivation,T7RNAP
6,GENE_deactivation,G_BNT_162b2


In [378]:
# How many species and reactions?
(ℳ, ℛ) = size(S);

In [379]:
# initialize -
species_index_table_data = Array{Any,2}(undef, ℳ, 2);

# build table -
for i ∈ 1:ℳ
    species_index_table_data[i,1] = i;
    species_index_table_data[i,2] = list_of_species[i];
end

# setup header -
species_index_header_table = (["Index", "Species"]);

# build table -
pretty_table(species_index_table_data; header=species_index_header_table);

┌───────┬───────────────────────────┐
│ Index │                   Species │
├───────┼───────────────────────────┤
│     1 │               G_BNT_162b2 │
│     2 │                    T7RNAP │
│     3 │                   M_atp_c │
│     4 │                   M_utp_c │
│     5 │                   M_ctp_c │
│     6 │                   M_gtp_c │
│     7 │            mRNA_BNT_162b2 │
│     8 │                   M_ppi_c │
│     9 │                   M_amp_c │
│    10 │                   M_ump_c │
│    11 │                   M_cmp_c │
│    12 │                   M_gmp_c │
│    13 │           T7RNAP_inactive │
│    14 │      G_BNT_162b2_inactive │
│    15 │ G_BNT_162b2_T7RNAP_closed │
│    16 │   G_BNT_162b2_T7RNAP_open │
└───────┴───────────────────────────┘


#### Setup the constants, feed rates and compositions

In [380]:
# how many chips in parallel?
number_of_chips = 15;

# what fraction of the mRNA degrades?
δ = 0.10;

# MSU split ratio -
θ = 0.95;

# Setup constants for transcription -
L = length(gene_sequence);
K = 0.116; # saturation constant units: μmol/L; Source: ACS Synth. Biol. 2018, 7, 8, 1844–1857 https://doi.org/10.1021/acssynbio.7b00465
v̇ₜ = (90.0)*(60); # units: nt/s; Source: BIND: 111871
u = 0.95; # u-factor; Source: ACS Synth. Biol. 2018, 7, 8, 1844–1857 https://doi.org/10.1021/acssynbio.7b00465

# volume -
V = 100.0*(1/1e6); # liquid reaction volume on each chip units: L

# Stock solution 1: DNA -> feed into splitter (mux) by pump 1
# Composition of the DNA source solution (flows into the chip in stream/channel 1)
G_BNT_162b2 = 2000.0;     # gene concentration in stock solution units: μmol/L

# Stock solution 2: PURExpress -> feed into splitter (mux) by pump 2
# Composition of PURExpress (flows into the chip in stream/channel 2)
T7RNAP = 100.0;          # concentration in PURExpress units: μmol/L
M_atp_c = 100*(1e6/1e3); # concentration in PURExpress units: μmol/L
M_utp_c = 100*(1e6/1e3); # concentration in PURExpress units: μmol/L
M_ctp_c = 100*(1e6/1e3); # concentration in PURExpress units: μmol/L
M_gtp_c = 100*(1e6/1e3); # concentration in PURExpress units: μmol/L

# Volumetric flow rates from the pump *into* the splitter unit - our base case will 1 ml/min into the chips, thus, we need
# to scale by the number of chips
V̇₁ = number_of_chips*100.0*(1/1e6); # volumetric flow rate of syringe pump 1 (this goes into splitter 1) units: L/min, must equal 300 mL/min or 0.3 L/min at most
V̇₂ = number_of_chips*3000.0*(1/1e6); # volumetric flow rate of syringe pump 2 (this goes into splitter 2) units: L/min, must equal 300 mL/min or 0.3 L/min

# stuff needed for later -
F̂₁ = V̇₁/(V̇₁+V̇₂); # do not change me
F̂₂ = V̇₂/(V̇₁+V̇₂); # do not change me

# stuff for the tables -
current_table_counter = 0; # do not change me

In [381]:
V̇₁
V̇₂

0.045

#### Specify the inputs streams

##### a) Specify the composition of feed stream 1
By default, this stream contains only the gene encoding the mRNA BNT-162b2 product. The mainstream is split into sub-streams that are fed into each chip.

In [382]:
# setup feed compostions for feed stream 1
ṅ₁ = zeros(ℳ); # default is zero, correct specific values -
ṅ₁[1] = G_BNT_162b2*V̇₁*(1/number_of_chips); # units: μmol/min

##### b) Specify the composition of feed stream 2
By default, feed stream 2 contains the [PURExpress](https://www.neb.com/products/e6800-purexpress-invitro-protein-synthesis-kit#Product%20Information), which has everything we need to make our mRNA product of interest _expect_ the linear DNA. The mainstream is split into sub-streams that are fed into each chip.

In [383]:
# setup feed compostions for feed stream 2
ṅ₂ = zeros(ℳ); # default is zero, then correct specific values -
ṅ₂[2] = T7RNAP*V̇₂*(1/number_of_chips);  # units: μmol/min
ṅ₂[3] = M_atp_c*V̇₂*(1/number_of_chips); # units: μmol/min
ṅ₂[4] = M_utp_c*V̇₂*(1/number_of_chips); # units: μmol/min
ṅ₂[5] = M_ctp_c*V̇₂*(1/number_of_chips); # units: μmol/min
ṅ₂[6] = M_gtp_c*V̇₂*(1/number_of_chips); # units: μmol/min

### Flux balance analysis setup
Flux balance analysis is a mathematical model and analysis approach that helps approximate cell-free reaction rates. This helps us estimate flows for different types of networks.To update the default species and flux bounds to model the process, we adjusted values of the volumetric flow rates of PURExpress and BNT-162b2 concentration in either syringe pumps. Modifying the volumetric flow rate of BNT-162b2 changes the GENE concentration. On the other hand, altering the PURExpress volumetric flow rate changes the RNAP concentration. Both of these changes affect the flux bounds, which allows us to model the process. 


#### Species bounds

In [385]:
# species bounds array -
species_bounds_array = [-(ṅ₁ .+ ṅ₂) 10000.0*ones(ℳ,1)];

#### Reaction bounds

In [386]:
# initialize -
flux_bounds_array = zeros(ℛ,2);
flux_bounds_array[:,2] .= 1000.0; # large default upper bound

# Get the RNAP and GENE concentration -
Rₜ = T7RNAP*F̂₂;        # effective RNAP concentratation on the chip for the bounds units: μmol/L
GENE = G_BNT_162b2*F̂₁ # effective GENE concentratation on the chip for the bounds units: μmol/L

# Setup bounds for transcription -
flux_bounds_array[3,:] .= Rₜ*(v̇ₜ/L)*u*(GENE/(K+GENE))*V; # equality constraint

# Setup bound for degradation (lower bound)
flux_bounds_array[4,1] = δ*flux_bounds_array[3,1];

In [387]:
# initialize -
reaction_index_table_data = Array{Any,2}(undef, ℛ, 4);

# build table -
for i ∈ 1:ℛ
    reaction_index_table_data[i,1] = i;
    reaction_index_table_data[i,2] = list_of_reactions[i];
    reaction_index_table_data[i,3] = flux_bounds_array[i,1];
    reaction_index_table_data[i,4] = flux_bounds_array[i,2];
end

# setup title string -
reaction_table_title = "Table $(current_table_counter+=1): Reaction index-name mapping table."

# setup header -
reaction_index_header_table = (["Index", "Reaction", "lower bound", "upper bound"], ["", "", "μmol/min", "μmol/min"]);

# build table -
pretty_table(reaction_index_table_data, title=reaction_table_title; header=reaction_index_header_table);

Table 1: Reaction index-name mapping table.
┌───────┬────────────────────────────┬─────────────┬─────────────┐
│ Index │                   Reaction │ lower bound │ upper bound │
│       │                            │    μmol/min │    μmol/min │
├───────┼────────────────────────────┼─────────────┼─────────────┤
│     1 │       TX_BNT_162b2_binding │         0.0 │      1000.0 │
│     2 │          TX_BNT_162b2_open │         0.0 │      1000.0 │
│     3 │    BNT_162b2_transcription │   0.0118697 │   0.0118697 │
│     4 │ mRNA_BNT_162b2_degradation │  0.00118697 │      1000.0 │
│     5 │          RNAP_deactivation │         0.0 │      1000.0 │
│     6 │          GENE_deactivation │         0.0 │      1000.0 │
└───────┴────────────────────────────┴─────────────┴─────────────┘


#### Objective coefficient array
* What is the objective you used?

In [54]:
# setup the objective coefficient array -
obj_vector = zeros(ℛ);
obj_vector[3] = -1; # why negative?
obj_vector

6-element Vector{Float64}:
  0.0
  0.0
 -1.0
  0.0
  0.0
  0.0

## Results and Discussion 

### Compute the optimal extent of reaction and exit stream composition per chip

In [389]:
# compute the optimal flux, and then estimate the output on the chip
result = compute_optimal_extent(S, flux_bounds_array, species_bounds_array, obj_vector);

# build a system stream table -
ϵ̇ = result.calculated_flux_array;

# compute the output -
ṅ₃ = (ṅ₁ + ṅ₂) + S*ϵ̇;

# compute the Δ reaction -
Δ = S*ϵ̇;

# get the exit and status flags to make sure all is ok
exit_flag = result.exit_flag;
status_flag = result.status_flag;

In [390]:
system_flux_table_data = Array{Any,2}(undef, ℳ, 6);

# populate the table -
for i ∈ 1:ℳ
    system_flux_table_data[i,1] = list_of_species[i];
    system_flux_table_data[i,2] = i;
    system_flux_table_data[i,3] = ṅ₁[i];
    system_flux_table_data[i,4] = ṅ₂[i];
    system_flux_table_data[i,5] = round(ṅ₃[i], digits=3);
    system_flux_table_data[i,6] = round(Δ[i], digits=3);
end

# title -
species_mft_table_title = "Table $(current_table_counter+=1): Single chip species mole flow rates table; Streams 1 and 2 are inputs, stream 3 is the chip output. 
The solver returned the exit flag = $(exit_flag) and status_flag = $(status_flag)"

# header -
state_table_header = (
    ["Species", "index i", "ṅ₁,ᵢ", "ṅ₂,ᵢ", "ṅ₃,ᵢ", "Δ"], 
    ["","","(μmol/min)", "(μmol/min)", "(μmol/min)", "(μmol/min)"]
);

# show -
pretty_table(system_flux_table_data, title=species_mft_table_title; header = state_table_header)

Table 2: Single chip species mole flow rates table; Streams 1 and 2 are inputs, stream 3 is the chip output. 
The solver returned the exit flag = 0 and status_flag = 5
┌───────────────────────────┬─────────┬────────────┬────────────┬────────────┬────────────┐
│                   Species │ index i │       ṅ₁,ᵢ │       ṅ₂,ᵢ │       ṅ₃,ᵢ │          Δ │
│                           │         │ (μmol/min) │ (μmol/min) │ (μmol/min) │ (μmol/min) │
├───────────────────────────┼─────────┼────────────┼────────────┼────────────┼────────────┤
│               G_BNT_162b2 │       1 │        0.2 │        0.0 │        0.2 │        0.0 │
│                    T7RNAP │       2 │        0.0 │        0.3 │        0.3 │        0.0 │
│                   M_atp_c │       3 │        0.0 │      300.0 │    290.528 │     -9.472 │
│                   M_utp_c │       4 │        0.0 │      300.0 │    288.083 │    -11.917 │
│                   M_ctp_c │       5 │        0.0 │      300.0 │    287.418 │    -12.582 │
│   

### Design a downstream seperation using Magical Seperator Units (MSU)

In [391]:
# build a downstream seperation process with this number of levels:
number_of_levels = 7; # includes zero

In [392]:
# initialize -
tmp_storage_dict = Dict{Int64, MSULatticeModel}();

# is_product_vector -
is_product_vector = zeros(ℳ);
is_product_vector[7] = 1;

# compute the composition array -
for i ∈ 1:ℳ

    if (is_product_vector[i] == 1)
        msu_lattice_model = build(MSULatticeModel; ṅₒ = (number_of_chips)*ṅ₃[i], L = number_of_levels , u = θ, d = (1 - θ));
    else
        msu_lattice_model = build(MSULatticeModel; ṅₒ = (number_of_chips)*ṅ₃[i], L = number_of_levels , u = (1 - θ), d = θ);
    end
    
    # grab -
    tmp_storage_dict[i] = msu_lattice_model;
end

# grab the leaves -
nodes_dict = build_nodes_dictionary(number_of_levels);
children_dict = build_children_dictionary(nodes_dict);
tree_leaves = nodes_dict[number_of_levels-1];

# build a composition array -
number_of_nodes = length(tree_leaves);
composition_array = Array{Float64,2}(undef, number_of_nodes, ℳ);
for i ∈ 1:ℳ
    data = tmp_storage_dict[i].data;
    for j ∈ 1:number_of_nodes
        composition_array[j,i] = data[tree_leaves[j]]
    end
end

# make a pretty table and show the leaves of the tree -

# initialize -
sep_tree_flow_table_data = Array{Any,2}(undef, ℳ, length(tree_leaves) + 3)
for i ∈ 1:ℳ
    sep_tree_flow_table_data[i,1] = list_of_species[i];
    sep_tree_flow_table_data[i,2] = i;
    sep_tree_flow_table_data[i,3] = (number_of_chips)*ṅ₃[i] # put node 0 in table -
        
    for j ∈ 1:length(tree_leaves)
        sep_tree_flow_table_data[i,3+j] = composition_array[j,i]
    end
end

# labels row -
label_row = Array{String,1}();
push!(label_row,"Species");
push!(label_row,"index i")
push!(label_row,"N0")
for j ∈ 1:length(tree_leaves)
    push!(label_row, "N$(tree_leaves[j])");
end

# units row -
units_row = Array{String,1}();
push!(units_row, "");
push!(units_row, "");
for j ∈ 1:length(tree_leaves)+1
    push!(units_row, "μmol/min");
end

# header -
sep_tree_flow_table_header = (label_row, units_row);

# set title -
title = "Table $(current_table_counter+=1): Magical Seperator Unit (MSU) flow table; N0 denotes the feed while N⋆ denotes the leaves of the tree."

# show -
pretty_table(sep_tree_flow_table_data, title=title; header=sep_tree_flow_table_header)

Table 3: Magical Seperator Unit (MSU) flow table; N0 denotes the feed while N⋆ denotes the leaves of the tree.
┌───────────────────────────┬─────────┬──────────┬────────────┬─────────────┬─────────────┬─────────────┬───────────┬────────────┬────────────┐
│                   Species │ index i │       N0 │        N22 │         N23 │         N24 │         N25 │       N26 │        N27 │        N28 │
│                           │         │ μmol/min │   μmol/min │    μmol/min │    μmol/min │    μmol/min │  μmol/min │   μmol/min │   μmol/min │
├───────────────────────────┼─────────┼──────────┼────────────┼─────────────┼─────────────┼─────────────┼───────────┼────────────┼────────────┤
│               G_BNT_162b2 │       1 │      3.0 │  4.6875e-8 │  8.90625e-7 │  1.69219e-5 │ 0.000321516 │ 0.0061088 │   0.116067 │    2.20528 │
│                    T7RNAP │       2 │      4.5 │ 7.03125e-8 │  1.33594e-6 │  2.53828e-5 │ 0.000482273 │ 0.0091632 │   0.174101 │    3.30791 │
│                   M_atp

In [393]:
# build mol frac composition table -

# construct mol frac array -
mol_frac_array = Array{Float64,2}(undef, ℳ, length(tree_leaves)+1);

# node 0 -
ṅ₃_total = sum(ṅ₃);
for i ∈ 1:ℳ
    mol_frac_array[i,1] = ṅ₃[i]*(1/ṅ₃_total);    
end

# get the sums along rows -
ṅ_total = sum(composition_array,dims = 2);
for node ∈ 1:length(tree_leaves)
    for i ∈ 1:ℳ
        mol_frac_array[i,node+1] = composition_array[node,i]*(1/ṅ_total[node]);
    end
end

# initialize -
sep_tree_mol_frac_table_data = Array{Any,2}(undef, ℳ, length(tree_leaves) + 3)
for i ∈ 1:ℳ
    sep_tree_mol_frac_table_data[i,1] = list_of_species[i];
    sep_tree_mol_frac_table_data[i,2] = i;
    sep_tree_mol_frac_table_data[i,3] = round(mol_frac_array[i,1], digits=4) # put node 0 in table -
        
    for j ∈ 1:length(tree_leaves)
        sep_tree_mol_frac_table_data[i, 3+j] = round(mol_frac_array[i,j+1], digits=4)
    end
end

# labels row -
label_mft_row = Array{String,1}();
push!(label_mft_row,"Species");
push!(label_mft_row,"index i")
push!(label_mft_row,"N0")
for j ∈ 1:length(tree_leaves)
    push!(label_mft_row, "N$(tree_leaves[j])");
end

# units row -
units_mft_row = Array{String,1}();
push!(units_mft_row, "");
push!(units_mft_row, "");
for j ∈ 1:length(tree_leaves)+1
    push!(units_mft_row, "mole frac");
end

# header -
sep_tree_mft_table_header = (label_mft_row, units_mft_row);

# set title -
title_mft = "Table $(current_table_counter+=1): Magical Seperator Unit (MSU) composition table."

# show -
pretty_table(sep_tree_mol_frac_table_data, title=title_mft; header=sep_tree_mft_table_header)

Table 4: Magical Seperator Unit (MSU) composition table.
┌───────────────────────────┬─────────┬───────────┬───────────┬───────────┬───────────┬───────────┬───────────┬───────────┬───────────┐
│                   Species │ index i │        N0 │       N22 │       N23 │       N24 │       N25 │       N26 │       N27 │       N28 │
│                           │         │ mole frac │ mole frac │ mole frac │ mole frac │ mole frac │ mole frac │ mole frac │ mole frac │
├───────────────────────────┼─────────┼───────────┼───────────┼───────────┼───────────┼───────────┼───────────┼───────────┼───────────┤
│               G_BNT_162b2 │       1 │    0.0002 │       0.0 │    0.0001 │    0.0002 │    0.0002 │    0.0002 │    0.0002 │    0.0002 │
│                    T7RNAP │       2 │    0.0002 │       0.0 │    0.0001 │    0.0002 │    0.0002 │    0.0002 │    0.0002 │    0.0002 │
│                   M_atp_c │       3 │     0.241 │    0.0006 │    0.1118 │    0.2402 │     0.241 │     0.241 │     0.241 │    

### Financial Analysis

If we ran the system for 1 year, we would have to sell our mRNA product at \\$37423.54. After 5 years, we would have to sell our mRNA at roughly \\$6895.50 each year. If we ran the system for 10 years, we would have to sell the mRNA at \\$6205.12 each year for the NPV to equal zero. 

To improve the financial performance of our design, we could reduce the magnitude of the volumetric flow rates and slightly increase the number of chips that we are using. Another way to reduce costs is to utilize a series configuration instead of a parallel configuration. Alternatively, we could use a binary separation system in order to recycle PURExpress, which is further explored into in the summary and conclusions section.

$$cost_{stream2} = 45000 * 525000 * 0.0095 = 224,694,000$$
$$CF_0 = -(15*100 + 2*1000 + 7*100 + 3*75 + cost_{stream2}) = -224,698,425$$
$$\text{number of moles} = m = 0.011 * 525600 * 0.9976 = 5767.72$$
$$D_{i}^{-1} = (1+.041)^i$$

1 Year: 
$$CF_1 = -\frac{CF_0}{D_1^{-1}m} = -\frac{-224,698,425}{(1.041)*5767.72} = 37423.54$$

5 Years:
$$CF_5 = -(15*100 + 2*1000 + 7*100 + 3*75 + 5cost_{stream2}) = -1.123 * 10^9$$
$$P_5 = -\frac{CF_5}{m\sum_{i=1}^{10}D_i^{-1}} = -\frac{CF_5}{5m\sum_{n=1}^{5}{1.041^n}} = 6895.50$$

10 Years:
$$CF_{10} =  -(15*100 + 2*1000 + 7*100 + 3*75 + 10cost_{stream2}) = -2.247 * 10^9$$
$$P_{10} = -\frac{CF_{10}}{m\sum_{i=1}^{10}D_i^{-1}} = -\frac{CF_{10}}{5m\sum_{n=1}^{10}{1.041^n}} = 6205.12$$

## Summary and Conclusions
   The method we used in order to produce 99% pure mRNA was trial and error. We would change a single variable, and then we would run the code to see how the output of mRNA was affected. Variables that we experimented with were the number of levels of separations for the MSUs, the efficiency of the MSU, and the volumetric flow rates of the two syringe pumps in order to see how everything was affected and what would produce the most mRNA. Once we discovered the optimal numbers for each of our variables, we scaled up our number chips to 15 so we hit our target production of 0.1 μmol/min with 99% purity. Once we achieved this goal, we further experimented to see how to best cut our cost. We did this by changing the efficiency of the MSUs and seeing if we still had a 99% pure mRNA product. If we didn’t, we would increase the level of splits and until we reached 99% purity and took into account whether or not it was cheaper. Our original system used an extremely high flow rate of 0.300 L/min for syringe pump 2, which was the maximum flow rate that our syringe pump could go. This exceedingly large flow rate uses a significantly greater amount of PURExpress, which happens to be our greatest expenditure. Thus, our project team was able to find a way to greatly decrease our flow rate to 0.045 L/min by increasing the number of reactors we have. While we originally had 8 levels of separation, we attempted to see if we could save costs by dropping the number of levels to 7 and found that we were able to maintain a purity above 99% with an increase in reactors. Other ways to try and improve costs would be to use a series configuration in our project instead of a parallel configuration because a series configuration requires less reactors, which would save hundreds of dollars in cost. Although our system uses a linear separation of MSU’s in order to save costs, another way to save costs would be to arrange the MSUs in a binary separation tree and recycle the bottom-most stream of nearly pure PURExpress, pass it through a heat exchanger, then bring the PURExpress back up to combine with stream 2. Our system produces 0.11 μmol/min of mRNA with 99.76% purity and is certainly a reasonable solution to be able to mass produce the Covid-19 vaccine. However, it could be more optimized in certain ways so that it is more cost effective and still produces the same amount of mRNA at the right purity. The volumetric flow rate could be decreased even more and the number of reactors increased again so we are not over using PURExpress. We could also optimize the reaction by lowering the efficiency of the MSUs to 90% and increasing the levels of splits since the cost of the MSUs with 90% efficiency is ⅕ of the price of MSUs with 95% efficiency. Therefore, there is potential to curb cost even further. 


## References and Additional Resources
* Put any references or additional citations here

Immunization Basics | CDC. [www.cdc.gov/vaccines/vac-gen/imz-basics.html](www.cdc.gov/vaccines/vac-gen/imz-basics.html)

New England Biolabs. PURExpress® in Vitro Protein Synthesis Kit | NEB. [www.neb.com/products/e6800-purexpress-invitro-protein-synthesis-kit](www.neb.com/products/e6800-purexpress-invitro-protein-synthesis-kit).

Office of Infectious Disease and HIV/AIDS Policy (OIDP). “Vaccines Protect You.” HHS.gov, 6 May 2022, [www.hhs.gov/immunization/basics/work/prevention/index.html](www.hhs.gov/immunization/basics/work/prevention/index.html).

“What’s Different About Messenger RNA (mRNA) Vaccines for COVID-19?” Memorial Sloan Kettering Cancer Center, [www.mskcc.org/coronavirus/what-s-different-about-messenger-rna-vaccines-covid-19](www.mskcc.org/coronavirus/what-s-different-about-messenger-rna-vaccines-covid-19).